In [3]:
# imports data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Загрузка и подготовка датафрейма

In [4]:
# Загружаем инстанс и посмотрим на описание
iris = load_iris()

print('Ключи:\n\t', iris.keys())
print('Признаки:\n\t', iris.feature_names)
print('Таргет:\n\t', list(zip(iris.target_names, np.unique(iris.target) )))

Ключи:
	 dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])
Признаки:
	 ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Таргет:
	 [('setosa', 0), ('versicolor', 1), ('virginica', 2)]


In [5]:
y = iris.target

np.unique(y)

array([0, 1, 2])

In [6]:
# Загрузим данные в табличное представление
df = pd.concat([
                pd.DataFrame(iris.data, columns=iris.feature_names),
                pd.DataFrame(iris.target, columns=['target'])
                ],
                 axis=1
                )
df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
# В данных оставляем только 2 класса: Iris Versicolor, Iris Virginica.
df = df[df['target'] != 0]
df.target.unique()

array([1, 2])

In [8]:
# Представим target в виде бинарной классификации 0 и 1 (Iris Versicolor, Iris Virginica.)
le = LabelEncoder()

le.fit(df.target)
df['target'] = le.fit_transform(df.target)
df.target.unique()

array([0, 1], dtype=int64)

In [9]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [10]:
# Переименуем колонки и посмотрим на подготовленные данные:
df = df.rename(columns={
                        'sepal length (cm)' : 'sepal_length','sepal width (cm)': 'sepal_width',
                        'petal length (cm)': 'petal_length', 'petal width (cm)': 'petal_width',})
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
50,7.0,3.2,4.7,1.4,0
51,6.4,3.2,4.5,1.5,0
52,6.9,3.1,4.9,1.5,0
53,5.5,2.3,4.0,1.3,0
54,6.5,2.8,4.6,1.5,0


In [11]:
df.shape

(100, 5)

In [12]:
X = df.drop('target', axis=1)
y = df['target']

# Построение модели логистической регрессии

In [21]:
class MyLogisticRegression:

    def __init__(self, lr=0.001, epoches=100000, fit_intercept=True, threshold=0.5):
        self.lr = lr
        self.epoches = epoches
        self.fit_intercept = fit_intercept
        self.threshold = threshold

   # Внутренние функции для реализации градиентного спуска

    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)

    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def __logg_loss(self, h, y):
        return (-y*np.log(h) - (1-y)*np.log(1-h)).mean()

# Внешние функции для пользователей

    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_prob(X) >= threshold


# Самая важная функция, которая производит обновление весов в сторону антиградиента 
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)

        # Инцализация весов, размерность вектора совпадает с количеством фитчей
        self.theta = np.zeros(X.shape[1])

        for epoch in range(self.epoches):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient

            if epoch % 10000 == 0:
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f' Entropy Log loss: {self.__logg_loss(h, y)}\t')
        
        def accuracy_score(self, X):
            return (self.predict(X) == y).mean()


In [22]:
# Создаю инстанс собственного класса логистической регрессии
model = MyLogisticRegression()

In [23]:
model.fit(X, y)

 Entropy Log loss: 0.6929842387518904	
 Entropy Log loss: 0.43756675783538646	
 Entropy Log loss: 0.33293547782308125	
 Entropy Log loss: 0.2780110351282435	
 Entropy Log loss: 0.2442889509360885	
 Entropy Log loss: 0.22139899810361097	
 Entropy Log loss: 0.20476934352984247	
 Entropy Log loss: 0.1920868877683369	
 Entropy Log loss: 0.18205799423968366	
 Entropy Log loss: 0.17390250844096955	


In [25]:
# Качество алгоритма

y_pred = model.predict(X)
score = (y_pred == y).mean()
print(f'Метрика качества accuracy: {score}')

Метрика качества accuracy: 0.97
